In [85]:
"""
Author
Johan Tejning  - jote16@student.bth.se
Tokunbo Ojo    -  tooj21@student.bth.se
"""

'\nAuthor\nJohan Tejning  - jote16@student.bth.se\nTokunbo Ojo    -  tooj21@student.bth.se\n'

In [32]:
#Import libraries
from scipy import stats
from scipy.stats import studentized_range
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import scipy.stats
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import time
import math

In [37]:
#Read spambase.data into pandas dataframe
df = pd.read_csv('spambase.data',header = None)
df

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3,88,0
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4,14,0
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6,118,0
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5,78,0


In [4]:
# split time serie into train and test
# x = df.iloc[:, 0:57]
# y = df[57]
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [5]:
# create discretizer for Equal Width Discretizer
# kbins = KBinsDiscretizer(n_bins=2, strategy='uniform', encode='ordinal')

In [26]:
# Equal-Width Discretization implementation
# def descretize(data):
#     frames = []
#     for i in range(0,57):
#         bin = kbins.fit_transform(np.array(data.iloc[:, i:i + 1]).reshape(-1,1))
#         frames.append(pd.DataFrame(bin, columns=[str(i)]))
#     return pd.concat(frames, axis=1)
# X_train = descretize(X_train)
# X_test = descretize(X_test)
# X_train

In [38]:
names = [
    "Nearest Neighbors",
    "Linear SVM",
   # "Decision Tree",
   # "Random Forest",
    "Naive Bayes"
]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
  #  DecisionTreeClassifier(max_depth=5),
  #  RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    GaussianNB()
]

In [39]:
# iterate over classifiers
performance = {"Nearest Neighbors" : [], "Linear SVM": [], "Naive Bayes": []}
accuracy = {"Nearest Neighbors" : [], "Linear SVM": [], "Naive Bayes": []}
f_measure = {"Nearest Neighbors" : [], "Linear SVM": [], "Naive Bayes": []}
fold = []

def train_model(train, test, fold_no):
    X_train = train.iloc[:, 0:57]
    y_train = train[57]
    X_test = test.iloc[:, 0:57]
    y_test = test[57]
    for name, clf in zip(names, classifiers):
        start = time.time()
        clf.fit(X_train, y_train)
        end = time.time()
        performance[name].append(end - start)
        y_pred = clf.predict(X_test) # make predictions on test set
        acc = accuracy_score(y_test.values, y_pred)
        accuracy[name].append(acc)
        f_score = f1_score(y_test.values, y_pred)
        f_measure[name].append(f_score)
    fold.append(fold_no)



In [40]:
# Initialise a StratifiedKFold object and declare the number of splits we want, which in our case is 10.
skf = StratifiedKFold(n_splits=10)
fold_no = 1

for train_index, test_index in skf.split(df, df[57]):
    train = df.loc[train_index,:]
    test = df.loc[test_index,:]
    train_model(train, test, fold_no)
    fold_no += 1


In [79]:
def calculate_rank(metrics):
    rank1 = []
    rank2 = []
    rank3 = []
    for index, row in metrics.iterrows():
        temp = row.copy()
        sorted = -np.sort(-temp.values)
        rank1.append(sorted.tolist().index(row[0]) + 1)
        rank2.append(sorted.tolist().index(row[1]) + 1)
        rank3.append(sorted.tolist().index(row[2]) + 1)

        result = {}
    result["Nearest Neighbors"] = rank1
    result["Linear SVM"] = rank2
    result["Naive Bayes"] = rank3
    ranking_df = pd.DataFrame.from_dict(result)
    return ranking_df

In [81]:
#1) computational performance in terms of training time,
training_time_metrics = pd.DataFrame(performance, index = fold)
training_time_metrics

,Nearest Neighbors,Linear SVM,Naive Bayes
1,0.005303,9.664300,0.004280
2,0.002474,13.350164,0.004630
3,0.001748,14.689101,0.004194
4,0.001662,6.764337,0.004360
5,0.001661,4.452735,0.004698
6,0.002009,14.721831,0.004494
7,0.001590,9.260936,0.004147
8,0.001652,9.320597,0.004821
9,0.002174,9.154779,0.004679
10,0.001652,9.983524,0.004074


In [82]:
#Calculate the ranks of training time metrics
training_time_metrics_ranking = calculate_rank(training_time_metrics)
training_time_metrics_ranking

,Nearest Neighbors,Linear SVM,Naive Bayes
0,2,1,3
1,3,1,2
2,3,1,2
3,3,1,2
4,3,1,2
5,3,1,2
6,3,1,2
7,3,1,2
8,3,1,2
9,3,1,2


In [86]:
#) predictive performance based on accuracy,
accuracy_metrics = pd.DataFrame(accuracy, index = fold)
accuracy_metrics

,Nearest Neighbors,Linear SVM,Naive Bayes
1,0.759219,0.928416,0.843818
2,0.780435,0.926087,0.863043
3,0.784783,0.897826,0.878261
4,0.800000,0.941304,0.867391
5,0.821739,0.845652,0.884783
6,0.821739,0.928261,0.828261
7,0.808696,0.956522,0.832609
8,0.815217,0.923913,0.867391
9,0.713043,0.858696,0.634783
10,0.736957,0.858696,0.717391


In [88]:
#Calculate the ranks of accuracy metrics
accuracy_metrics_ranking = calculate_rank(accuracy_metrics)
accuracy_metrics_ranking

,Nearest Neighbors,Linear SVM,Naive Bayes
0,3,1,2
1,3,1,2
2,3,1,2
3,3,1,2
4,3,2,1
5,3,1,2
6,3,1,2
7,3,1,2
8,2,1,3
9,2,1,3


In [90]:
#3) predictive performance based on F-measure.
f_measure_metrics = pd.DataFrame(f_measure, index = fold)
f_measure_metrics

,Nearest Neighbors,Linear SVM,Naive Bayes
1,0.687324,0.907042,0.829384
2,0.718663,0.905556,0.849642
3,0.717949,0.869806,0.864078
4,0.732558,0.925620,0.853717
5,0.772222,0.817481,0.865823
6,0.781915,0.911051,0.819222
7,0.733333,0.942857,0.820513
8,0.763231,0.901408,0.847880
9,0.659794,0.826667,0.669291
10,0.684073,0.815864,0.711111


In [91]:
#Calculate the ranks of f measure metrics
f_measure_metrics_ranking = calculate_rank(f_measure_metrics)
f_measure_metrics_ranking

,Nearest Neighbors,Linear SVM,Naive Bayes
0,3,1,2
1,3,1,2
2,3,1,2
3,3,1,2
4,3,2,1
5,3,1,2
6,3,1,2
7,3,1,2
8,3,1,2
9,3,1,2


In [92]:
rank_avg_training_time_metrics = training_time_metrics_ranking.mean(axis=0)
rank_avg_training_time_metrics

Nearest Neighbors    2.9
Linear SVM           1.0
Naive Bayes          2.1
dtype: float64

In [94]:
rank_avg_accuracy_metrics = accuracy_metrics_ranking.mean(axis=0)
rank_avg_accuracy_metrics

Nearest Neighbors    2.8
Linear SVM           1.1
Naive Bayes          2.1
dtype: float64

In [97]:
rank_avg_f_measure_metrics = f_measure_metrics_ranking.mean(axis=0)
rank_avg_f_measure_metrics

Nearest Neighbors    3.0
Linear SVM           1.1
Naive Bayes          1.9
dtype: float64

In [103]:
#Finding the sum of squared differences
def sum_of_squared_differences_1(rank_avg):
    sum = 0
    for x in rank_avg.values:
        sum = sum + (x - 2) ** 2
    sum_of_squared_differences_1 = 10 * sum
    return sum_of_squared_differences_1

In [104]:
#Finding the sum of squared differences 2
def sum_of_squared_differences_2(ranking_df):
    summation = 0
    for index, row in ranking_df.iterrows():
        summation = summation + ((row[0] - 2) ** 2) + ((row[1] - 2) ** 2) + ((row[2] - 2) ** 2)
    sum_of_squared_differences_2 = summation * 0.05
    return sum_of_squared_differences_2

In [105]:
#Finding p-value
p_value = scipy.stats.chi2.ppf(0.95, 3)
p_value

7.814727903251179

In [106]:
def friedman_statistic(rank_avg, ranking_df):
    friedman_statistic = sum_of_squared_differences_1(rank_avg) / sum_of_squared_differences_2(ranking_df)
    decision = "greater than " if friedman_statistic > p_value else "less than "
    decision2 = "can " if friedman_statistic > p_value else "can no "
    print("Since the Friedman statistic " + str(friedman_statistic) + " is " + decision + str(p_value) + " so we " + decision2 + "reject the null hypothesis that all algorithms perform equally")

In [107]:
# Training time metrics Friedman statistic
friedman_statistic(rank_avg_training_time_metrics, training_time_metrics_ranking)

Since the Friedman statistic 18.2 is greater than 7.814727903251179 so we can reject the null hypothesis that all algorithms perform equally


In [108]:
# Accuracy metrics Friedman statistic
friedman_statistic(rank_avg_accuracy_metrics, accuracy_metrics_ranking)

Since the Friedman statistic 14.599999999999994 is greater than 7.814727903251179 so we can reject the null hypothesis that all algorithms perform equally


In [109]:
# F Measure metrics Friedman statistic
friedman_statistic(rank_avg_f_measure_metrics, f_measure_metrics_ranking)

Since the Friedman statistic 18.2 is greater than 7.814727903251179 so we can reject the null hypothesis that all algorithms perform equally


In [110]:
#Finding critical difference (CD) using Nemenyi test
#The q alpha 2.343 is taken from the course textbook and also from this link https://plos.figshare.com/articles/dataset/Critical_values_for_the_two-tailed_Nemenyi_test_/7299293/1
#k = 3, n =  10

cd = 2.343 * math.sqrt(3 * (3 + 1) / (6 * 10))
cd

1.0478214542564015

In [111]:
#Nearest Neighbors - rank_avg[0]
#Linear SVM - rank_avg[1]
#Naive Bayes - rank_avg[2]
def critical_difference(rank_avg):
    no_of_means__of_ranks = len(rank_avg)
    for i in range(no_of_means__of_ranks - 1):
        for j in range(i + 1, no_of_means__of_ranks):
            diff = abs(rank_avg[i] - rank_avg[j])
            decision = " and it exceeds the critical difference " if diff > cd else " and does not exceeds the critical difference "
            print("The difference of " + rank_avg.index[i] + " - " + str(rank_avg[i])  + " and " + rank_avg.index[j] + " - " + str(rank_avg[j]) + " is " + str(diff) +  decision + str(cd))


In [112]:
# Training time metrics Nemenyi test
critical_difference(rank_avg_training_time_metrics)

The difference of Nearest Neighbors - 2.9 and Linear SVM - 1.0 is 1.9 and it exceeds the critical difference 1.0478214542564015
The difference of Nearest Neighbors - 2.9 and Naive Bayes - 2.1 is 0.7999999999999998 and does not exceeds the critical difference 1.0478214542564015
The difference of Linear SVM - 1.0 and Naive Bayes - 2.1 is 1.1 and it exceeds the critical difference 1.0478214542564015


In [113]:
# Accuracy metrics Nemenyi test
critical_difference(rank_avg_accuracy_metrics)

The difference of Nearest Neighbors - 2.8 and Linear SVM - 1.1 is 1.6999999999999997 and it exceeds the critical difference 1.0478214542564015
The difference of Nearest Neighbors - 2.8 and Naive Bayes - 2.1 is 0.6999999999999997 and does not exceeds the critical difference 1.0478214542564015
The difference of Linear SVM - 1.1 and Naive Bayes - 2.1 is 1.0 and does not exceeds the critical difference 1.0478214542564015


In [114]:
# F Measure Nemenyi test
critical_difference(rank_avg_f_measure_metrics)

The difference of Nearest Neighbors - 3.0 and Linear SVM - 1.1 is 1.9 and it exceeds the critical difference 1.0478214542564015
The difference of Nearest Neighbors - 3.0 and Naive Bayes - 1.9 is 1.1 and it exceeds the critical difference 1.0478214542564015
The difference of Linear SVM - 1.1 and Naive Bayes - 1.9 is 0.7999999999999998 and does not exceeds the critical difference 1.0478214542564015
